# Limpeza e Manutenção dos Dados - DATASETS: ureia.csv e creatinina.csv

## Índice

- [Link para retornar ao notebook principal](#retornar-para-notebook-principal---mainipynb)
- [Importando bibliotecas e pacotes](#importando-bibliotecas-e-pacotes)
- [Importando datasets](#importando-datasets)
- [Breve apresentação dos dados](#breve-apresentacao-dos-dados)
- [Realizando a análise exploratória](#analise-exploratoria-dos-dados)
- [Concatenando os datasets](#concatenando-os-datasets---ureia7-dias--creatinina7-dias)

## Retornar para notebook principal - main.ipynb

[Link para notebook principal](./main.ipynb)

## Importando bibliotecas e pacotes

In [1]:
from utils.alterar_dataset import preencher_dias_faltantes,\
    pivotear_dataset_ureia, pivotear_dataset_creatinina,\
    concatenar_dois_datasets_por_paciente, deletar_valores_absurdos,\
    preencher_valores_faltantes_linha
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

## Importando datasets

In [10]:
df_ureia = pd.read_csv("../databases/raw/ureia.csv", sep = ",", index_col = "subject_id")
df_creatinina = pd.read_csv("../databases/raw/creatinina.csv", sep = ",", index_col = "subject_id")
df_ureia_15_dias = pd.read_csv("../databases/raw/Ureia15dias.csv", sep = ",", index_col = "subject_id")
df_creatinina_15_dias = pd.read_csv("../databases/raw/Creatinina15dias.csv", sep = ",", index_col = "subject_id")

## Breve apresentacao dos dados

### DATASET ureia (7-dias)

In [4]:
df_ureia.head(10)

,day,UreiaMaxDia
subject_id,,
15158531,1,107.0
15158531,2,100.0
15158531,3,78.0
15158531,4,59.0
15158531,15,59.0
15158531,16,59.0
15158531,17,59.0
15158531,18,64.0
14800685,2,3.0


In [5]:
df_ureia.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42392 entries, 15158531 to 15274195
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   day          42392 non-null  int64  
 1   UreiaMaxDia  42392 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 993.6 KB


### DATASET creatinina (7-dias)

In [6]:
df_creatinina.head(10)

,day,CreatininaMaxDia
subject_id,,
15158531,15,3.8
15158531,16,3.9
15158531,17,3.9
15158531,18,3.7
13697731,66,0.1
13697731,68,0.1
15796335,5,4.2
15796335,6,5.4
15796335,7,4.2


In [7]:
df_creatinina.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42383 entries, 15158531 to 12953561
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   day               42383 non-null  int64  
 1   CreatininaMaxDia  42383 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 993.4 KB


### DATASET ureia (15-dias)

In [4]:
df_ureia_15_dias.head(10)

,day,UreiaMaxDia
subject_id,,
10032381,1,9.0
10032381,2,9.0
10032381,3,9.0
10032381,4,14.0
10032381,5,21.0
10032381,6,24.0
10032381,7,26.0
10032381,8,29.0
10032381,9,29.0


In [5]:
df_ureia_15_dias.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14520 entries, 10032381 to 19995595
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   day          14520 non-null  int64  
 1   UreiaMaxDia  14245 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 340.3 KB


### DATASET creatinina (15-dias)

In [11]:
df_creatinina_15_dias.head(10)

,day,CreatininaMaxDia
subject_id,,
10032381,1,1.3
10032381,2,1.2
10032381,3,1.2
10032381,4,1.2
10032381,5,1.2
10032381,6,1.2
10032381,7,1.2
10032381,8,1.2
10032381,9,1.2


In [12]:
df_creatinina_15_dias.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13307 entries, 10032381 to 19995595
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   day               13307 non-null  int64  
 1   CreatininaMaxDia  13307 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 311.9 KB


## Limpeza

### Retirando todos os dados cujo dia (coluna *day*) é maior que `7`

#### DATASET ureia (7-dias)

In [8]:
df_ureia_dia_menor_7 = df_ureia[df_ureia["day"] < 8]
df_ureia_dia_menor_7["day"].unique()

array([1, 2, 3, 4, 5, 6, 7])

In [9]:
df_ureia_dia_menor_7["day"].value_counts()

day
1    3297
3    3281
2    3277
4    3267
5    3262
6    3242
7    3082
Name: count, dtype: int64

#### DATASET creatinina (7-dias)

In [10]:
df_creatinina_dia_menor_7 = df_creatinina[df_creatinina["day"] < 8]
df_creatinina_dia_menor_7["day"].unique()

array([5, 6, 7, 4, 3, 2, 1])

In [11]:
df_creatinina_dia_menor_7["day"].value_counts()

day
1    3298
3    3282
2    3275
4    3267
5    3262
6    3241
7    3081
Name: count, dtype: int64

### Inserindo novos dias ate que todos os pacientes possuam 7 dias de observacao

#### DATASET ureia (7-dias)

In [12]:
df_ureia_dia_menor_7

,day,UreiaMaxDia
subject_id,,
15158531,1,107.0
15158531,2,100.0
15158531,3,78.0
15158531,4,59.0
14800685,2,3.0
...,...,...
13105864,3,57.0
16724979,7,57.0
16930541,5,57.0


In [13]:
df_ureia_dia_menor_7 = df_ureia_dia_menor_7.reset_index()
df_ureia_processed = preencher_dias_faltantes(df_ureia_dia_menor_7, ["subject_id", "day"], "UreiaMaxDia", 8)
df_ureia_processed = df_ureia_processed.reset_index()
df_ureia_processed.head(14)

,subject_id,day,UreiaMaxDia
0,10001884,1,30.0
1,10001884,2,NaN
2,10001884,3,38.0
3,10001884,4,31.0
4,10001884,5,26.0
5,10001884,6,22.0
6,10001884,7,14.0
7,10004422,1,21.0
8,10004422,2,13.0
9,10004422,3,17.0


#### DATASET creatinina (7-dias)

In [14]:
df_creatinina_dia_menor_7 = df_creatinina_dia_menor_7.reset_index()
df_creatinina_processed = preencher_dias_faltantes(df_creatinina_dia_menor_7, ["subject_id", "day"], "CreatininaMaxDia", 8)
df_creatinina_processed = df_creatinina_processed.reset_index()
df_creatinina_processed.head(14)

,subject_id,day,CreatininaMaxDia
0,10001884,1,1.1
1,10001884,2,NaN
2,10001884,3,1.3
3,10001884,4,0.9
4,10001884,5,0.8
5,10001884,6,0.6
6,10001884,7,0.5
7,10004422,1,0.7
8,10004422,2,0.7
9,10004422,3,0.7


#### Salvando os datasets processados

In [15]:
df_ureia_processed.to_csv("../databases/processed/ureia_processed.csv", sep = ",")
df_creatinina_processed.to_csv("../databases/processed/creatinina_processed.csv", sep = ",")

### Pivoteando os datasets

#### DATASET ureia (7-dias)

In [16]:
df_ureia_processed = pivotear_dataset_ureia(df_ureia_processed, "subject_id", "day", "UreiaMaxDia")
df_ureia_processed

,ureia_dia_1,ureia_dia_2,ureia_dia_3,ureia_dia_4,ureia_dia_5,ureia_dia_6,ureia_dia_7
subject_id,,,,,,,
10001884,30.0,NaN,38.0,31.0,26.0,22.0,14.0
10004422,21.0,13.0,17.0,16.0,20.0,24.0,NaN
10004733,44.0,41.0,39.0,37.0,28.0,26.0,28.0
10005606,6.0,6.0,10.0,6.0,11.0,NaN,15.0
10010867,8.0,5.0,5.0,5.0,6.0,7.0,9.0
...,...,...,...,...,...,...,...
19990427,45.0,43.0,26.0,21.0,30.0,28.0,27.0
19995595,21.0,17.0,23.0,21.0,23.0,33.0,39.0
19999068,3.0,2.0,2.0,5.0,6.0,6.0,NaN


#### DATASET creatinina (7-dias)

In [17]:
df_creatinina_processed = pivotear_dataset_creatinina(df_creatinina_processed, "subject_id", "day", "CreatininaMaxDia")
df_creatinina_processed

,creatinina_dia_1,creatinina_dia_2,creatinina_dia_3,creatinina_dia_4,creatinina_dia_5,creatinina_dia_6,creatinina_dia_7
subject_id,,,,,,,
10001884,1.1,NaN,1.3,0.9,0.8,0.6,0.5
10004422,0.7,0.7,0.7,0.6,0.6,0.7,NaN
10004733,3.4,3.8,3.7,3.4,2.6,2.6,2.7
10005606,0.7,0.6,0.5,0.4,0.5,NaN,0.5
10010867,0.7,0.7,0.5,0.3,0.4,0.5,0.5
...,...,...,...,...,...,...,...
19990427,1.7,1.3,0.7,0.5,0.7,0.6,0.6
19995595,1.6,1.1,1.7,1.5,1.4,1.8,1.6
19999068,0.7,0.5,0.7,0.7,0.6,0.7,NaN


#### DATASET ureia (15-dias)

In [8]:
df_ureia_15_dias.reset_index(inplace = True)
df_ureia_15_dias_processed = pivotear_dataset_ureia(df_ureia_15_dias, "subject_id", "day", "UreiaMaxDia")
df_ureia_15_dias_processed

,ureia_dia_1,ureia_dia_2,ureia_dia_3,ureia_dia_4,ureia_dia_5,ureia_dia_6,ureia_dia_7,ureia_dia_8,ureia_dia_9,ureia_dia_10,ureia_dia_11,ureia_dia_12,ureia_dia_13,ureia_dia_14,ureia_dia_15
subject_id,,,,,,,,,,,,,,,
10032381,9.0,9.0,9.0,14.0,21.0,24.0,26.0,29.0,29.0,28.0,26.0,28.0,28.0,30.0,29.0
10038688,26.0,27.0,29.0,29.0,28.0,31.0,35.0,36.0,37.0,38.0,37.0,36.0,37.0,36.0,35.0
10058575,9.0,11.0,10.0,9.0,11.0,13.0,14.0,15.0,17.0,19.0,20.0,23.0,26.0,24.0,26.0
10064854,28.0,30.0,31.0,43.0,49.0,65.0,81.0,100.0,121.0,140.0,146.0,139.0,116.0,84.0,54.0
10109956,26.0,23.0,22.0,27.0,45.0,49.0,53.0,51.0,44.0,50.0,65.0,64.0,67.0,62.0,54.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19950864,45.0,35.0,30.0,32.0,36.0,32.0,38.0,37.0,36.0,34.0,31.0,25.0,23.0,21.0,21.0
19957285,12.0,8.0,NaN,13.0,16.0,11.0,NaN,12.0,17.0,NaN,23.0,26.0,29.0,31.0,NaN
19965625,34.0,32.0,34.0,36.0,35.0,33.0,37.0,38.0,46.0,49.0,50.0,53.0,50.0,50.0,58.0


#### DATASET cretinina (15-dias)

In [19]:
df_creatinina_15_dias.reset_index(inplace = True)
df_creatinina_15_dias_processed = pivotear_dataset_creatinina(df_creatinina_15_dias, "subject_id", "day", "CreatininaMaxDia")
df_creatinina_15_dias_processed

,creatinina_dia_1,creatinina_dia_2,creatinina_dia_3,creatinina_dia_4,creatinina_dia_5,creatinina_dia_6,creatinina_dia_7,creatinina_dia_8,creatinina_dia_9,creatinina_dia_10,creatinina_dia_11,creatinina_dia_12,creatinina_dia_13,creatinina_dia_14,creatinina_dia_15
subject_id,,,,,,,,,,,,,,,
10032381,1.3,1.2,1.2,1.2,1.2,1.2,1.2,1.2,1.2,1.2,1.0,1.0,1.1,1.2,1.2
10038688,1.6,1.9,1.9,2.0,1.5,1.5,1.6,1.6,1.8,1.6,1.4,1.3,1.3,1.3,1.1
10058575,0.8,0.7,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.7,0.6,0.6,0.6,0.5,0.6
10064854,0.7,0.6,1.1,1.3,1.4,1.8,2.2,2.9,3.2,3.6,3.7,3.3,2.5,1.7,1.1
10109956,1.6,1.5,1.4,1.6,2.3,2.1,2.2,2.0,1.8,2.0,2.1,2.1,2.1,2.0,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19942150,0.9,0.6,0.5,0.7,0.6,0.7,0.7,0.6,0.8,1.1,1.2,1.4,1.5,0.6,0.7
19950864,1.4,1.2,0.9,0.9,1.1,0.8,0.9,0.8,0.9,0.7,0.9,0.8,0.7,0.9,0.7
19965625,1.8,1.8,1.9,1.9,1.8,1.5,1.4,1.2,1.4,1.2,1.3,1.2,1.1,0.9,1.0


#### Salvando os datasets pivoteados

In [20]:
df_ureia_processed.to_csv("../databases/processed/ureia_pivoted.csv", sep = ",")
df_creatinina_processed.to_csv("../databases/processed/creatinina_pivoted.csv", sep = ",")
df_ureia_15_dias_processed.to_csv("../databases/processed/ureia_15_dias_pivoted.csv", sep = ",")
df_creatinina_15_dias_processed.to_csv("../databases/processed/creatinina-15_dias_pivoted.csv", sep = ",")

## Concatenando os datasets - ureia(7 dias) & creatinina(7 dias)

#### Removendo valores outliers do dataset e os substituindo por `np.nan`

In [18]:
df_ureia_processed = deletar_valores_absurdos(df_ureia_processed, df_ureia_processed.max().max())
df_ureia_processed.max()

ureia_dia_1    212.0
ureia_dia_2    162.0
ureia_dia_3    171.0
ureia_dia_4    184.0
ureia_dia_5    182.0
ureia_dia_6    206.0
ureia_dia_7    195.0
dtype: object

In [19]:
df_creatinina_processed = deletar_valores_absurdos(df_creatinina_processed, df_creatinina_processed.max().max())
df_creatinina_processed.max()

creatinina_dia_1    117.0
creatinina_dia_2     16.0
creatinina_dia_3     15.0
creatinina_dia_4     14.6
creatinina_dia_5     14.7
creatinina_dia_6     15.2
creatinina_dia_7     15.6
dtype: object

#### Iterando pelo dataset e inserindo dados de dias anteriores em dias posteriores que contém dados nulos

In [20]:
df_ureia_processed = preencher_valores_faltantes_linha(df_ureia_processed)
df_ureia_processed.head(10)

/home/igorsantana/Documents/trabalho/TCC/src/utils/alterar_dataset.py:69: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_transposto[coluna] = df_transposto[coluna].fillna(


,ureia_dia_1,ureia_dia_2,ureia_dia_3,ureia_dia_4,ureia_dia_5,ureia_dia_6,ureia_dia_7
subject_id,,,,,,,
10001884,30.0,30.0,38.0,31.0,26.0,22.0,14.0
10004422,21.0,13.0,17.0,16.0,20.0,24.0,24.0
10004733,44.0,41.0,39.0,37.0,28.0,26.0,28.0
10005606,6.0,6.0,10.0,6.0,11.0,11.0,15.0
10010867,8.0,5.0,5.0,5.0,6.0,7.0,9.0
10019777,17.0,22.0,16.0,19.0,18.0,17.0,16.0
10020944,34.0,28.0,26.0,22.0,21.0,20.0,19.0
10023486,42.0,48.0,54.0,63.0,68.0,69.0,70.0
10032381,9.0,9.0,9.0,14.0,21.0,24.0,26.0


In [22]:
df_creatinina_processed = preencher_valores_faltantes_linha(df_creatinina_processed)
df_creatinina_processed.head(10)

,creatinina_dia_1,creatinina_dia_2,creatinina_dia_3,creatinina_dia_4,creatinina_dia_5,creatinina_dia_6,creatinina_dia_7
subject_id,,,,,,,
10001884,1.1,1.1,1.3,0.9,0.8,0.6,0.5
10004422,0.7,0.7,0.7,0.6,0.6,0.7,0.7
10004733,3.4,3.8,3.7,3.4,2.6,2.6,2.7
10005606,0.7,0.6,0.5,0.4,0.5,0.5,0.5
10010867,0.7,0.7,0.5,0.3,0.4,0.5,0.5
10019777,0.9,0.7,0.9,0.8,0.7,0.6,0.5
10020944,1.4,1.6,1.8,1.6,1.5,1.8,1.6
10023486,3.1,3.4,3.2,3.4,2.8,2.2,1.9
10032381,1.3,1.2,1.2,1.2,1.2,1.2,1.2


In [23]:
df_ureia_processed.isna().sum()

ureia_dia_1    21
ureia_dia_2     2
ureia_dia_3     5
ureia_dia_4     5
ureia_dia_5     7
ureia_dia_6    12
ureia_dia_7    32
dtype: int64

In [24]:
df_creatinina_processed.isna().sum()

creatinina_dia_1    17
creatinina_dia_2     2
creatinina_dia_3     1
creatinina_dia_4     4
creatinina_dia_5     5
creatinina_dia_6     4
creatinina_dia_7    10
dtype: int64

#### Deletando linhas que contém valores nulos - mesmo depois da inserção de dados artificiais

##### Ureia

In [25]:
df_ureia_processed[df_ureia_processed.isna().any(axis = 1)].index

Index([10344732, 10463546, 10477920, 10553635, 10583686, 10615339, 10996857,
       11542534, 11611745, 11638303, 11663336, 11833476, 12344021, 12374214,
       12606435, 12934260, 13679248, 13681485, 13824877, 13827765, 13911200,
       14363068, 14383658, 14623418, 14691603, 14696918, 14785854, 14816630,
       15090519, 15158294, 15204620, 15409850, 15455733, 15583807, 15640315,
       15763754, 15993533, 16327028, 16832227, 17131210, 17327554, 17484283,
       17635990, 17718694, 17803326, 18135694, 18152377, 18213765, 18560897,
       18638524, 18757959, 18976063, 19392949, 19571102, 19571265, 19694231],
      dtype='int64', name='subject_id')

In [26]:
df_ureia_processed = df_ureia_processed.dropna(axis = 0)
df_ureia_processed.isna().sum().sum()

0

##### Creatinina

In [27]:
df_creatinina_processed[df_creatinina_processed.isna().any(axis = 1)].index

Index([10463546, 10553635, 10996857, 11611745, 11663336, 11833476, 12344021,
       12374214, 12934260, 13681485, 13824877, 14383658, 14691603, 14816630,
       15409850, 15583807, 15640315, 15993533, 16327028, 17327554, 17484283,
       17635990, 17718694, 17803326, 18135694, 18152377, 18560897, 18638524,
       18757959],
      dtype='int64', name='subject_id')

In [28]:
df_creatinina_processed = df_creatinina_processed.dropna(axis = 0)
df_creatinina_processed.isna().sum().sum()

0

### Realizando a concatenação

In [29]:
df_concatenado = concatenar_dois_datasets_por_paciente(df_esquerda = df_ureia_processed,
                                                       df_direita = df_creatinina_processed)

df_concatenado.head(10)

,ureia_dia_1,ureia_dia_2,ureia_dia_3,ureia_dia_4,ureia_dia_5,ureia_dia_6,ureia_dia_7,creatinina_dia_1,creatinina_dia_2,creatinina_dia_3,creatinina_dia_4,creatinina_dia_5,creatinina_dia_6,creatinina_dia_7
subject_id,,,,,,,,,,,,,,
10001884,30.0,30.0,38.0,31.0,26.0,22.0,14.0,1.1,1.1,1.3,0.9,0.8,0.6,0.5
10004422,21.0,13.0,17.0,16.0,20.0,24.0,24.0,0.7,0.7,0.7,0.6,0.6,0.7,0.7
10004733,44.0,41.0,39.0,37.0,28.0,26.0,28.0,3.4,3.8,3.7,3.4,2.6,2.6,2.7
10005606,6.0,6.0,10.0,6.0,11.0,11.0,15.0,0.7,0.6,0.5,0.4,0.5,0.5,0.5
10010867,8.0,5.0,5.0,5.0,6.0,7.0,9.0,0.7,0.7,0.5,0.3,0.4,0.5,0.5
10019777,17.0,22.0,16.0,19.0,18.0,17.0,16.0,0.9,0.7,0.9,0.8,0.7,0.6,0.5
10020944,34.0,28.0,26.0,22.0,21.0,20.0,19.0,1.4,1.6,1.8,1.6,1.5,1.8,1.6
10023486,42.0,48.0,54.0,63.0,68.0,69.0,70.0,3.1,3.4,3.2,3.4,2.8,2.2,1.9
10032381,9.0,9.0,9.0,14.0,21.0,24.0,26.0,1.3,1.2,1.2,1.2,1.2,1.2,1.2


### Verificando se concatenação gerou valores NaN

In [30]:
df_concatenado.isna().sum()

ureia_dia_1         27
ureia_dia_2         27
ureia_dia_3         27
ureia_dia_4         27
ureia_dia_5         27
ureia_dia_6         27
ureia_dia_7         27
creatinina_dia_1     0
creatinina_dia_2     0
creatinina_dia_3     0
creatinina_dia_4     0
creatinina_dia_5     0
creatinina_dia_6     0
creatinina_dia_7     0
dtype: int64

### Pegando os `id`s dos pacientes que não possuem leitura de ureia, mas possuem leitura de creatinina

In [31]:
df_concatenado[df_concatenado.isna().any(axis = 1)].index

Index([10344732, 10477920, 10583686, 10615339, 11542534, 11638303, 12606435,
       13679248, 13827765, 13911200, 14363068, 14623418, 14696918, 14785854,
       15090519, 15158294, 15204620, 15455733, 15763754, 16832227, 17131210,
       18213765, 18976063, 19392949, 19571102, 19571265, 19694231],
      dtype='int64', name='subject_id')

### Removendo os pacientes que não possuem leitura de ureia, mas possuem leitura de creatinina

In [32]:
df_concatenado = df_concatenado.dropna(axis = 0)
df_concatenado.isna().sum().sum()

0

### Salvando o dataset concatenado

In [33]:
df_concatenado.to_csv("../databases/processed/ureia_creatinina_7_dias_concatenado.csv", sep = ",")

## Concatenando os datasets - ureia(15-dias) & creatinina(15-dias)

#### Removendo valores outliers do dataset e os substituindo por `np.nan`

In [16]:
df_ureia_15_dias_processed.max()

ureia_dia_1        148.0
ureia_dia_2        138.0
ureia_dia_3        145.0
ureia_dia_4        129.0
ureia_dia_5     999999.0
ureia_dia_6        150.0
ureia_dia_7        185.0
ureia_dia_8        153.0
ureia_dia_9        166.0
ureia_dia_10       180.0
ureia_dia_11       176.0
ureia_dia_12       197.0
ureia_dia_13       229.0
ureia_dia_14       200.0
ureia_dia_15       181.0
dtype: float64

In [17]:
df_ureia_15_dias_processed = deletar_valores_absurdos(df_ureia_15_dias_processed, df_ureia_15_dias_processed.max().max())
df_ureia_15_dias_processed.max()

ureia_dia_1     148.0
ureia_dia_2     138.0
ureia_dia_3     145.0
ureia_dia_4     129.0
ureia_dia_5     149.0
ureia_dia_6     150.0
ureia_dia_7     185.0
ureia_dia_8     153.0
ureia_dia_9     166.0
ureia_dia_10    180.0
ureia_dia_11    176.0
ureia_dia_12    197.0
ureia_dia_13    229.0
ureia_dia_14    200.0
ureia_dia_15    181.0
dtype: float64

In [21]:
df_creatinina_15_dias_processed.max()

creatinina_dia_1     117.0
creatinina_dia_2      13.0
creatinina_dia_3      12.6
creatinina_dia_4      11.8
creatinina_dia_5      11.9
creatinina_dia_6       9.1
creatinina_dia_7      11.1
creatinina_dia_8      10.7
creatinina_dia_9      11.1
creatinina_dia_10     10.0
creatinina_dia_11      8.1
creatinina_dia_12     12.7
creatinina_dia_13     13.5
creatinina_dia_14     12.5
creatinina_dia_15     10.8
dtype: float64

#### Iterando pelo dataset e inserindo dados de dias anteriores em dias posteriores que contém dados nulos

In [22]:
df_ureia_15_dias_processed = preencher_valores_faltantes_linha(df_ureia_15_dias_processed)
df_ureia_15_dias_processed.head(10)

,ureia_dia_1,ureia_dia_2,ureia_dia_3,ureia_dia_4,ureia_dia_5,ureia_dia_6,ureia_dia_7,ureia_dia_8,ureia_dia_9,ureia_dia_10,ureia_dia_11,ureia_dia_12,ureia_dia_13,ureia_dia_14,ureia_dia_15
subject_id,,,,,,,,,,,,,,,
10032381,9.0,9.0,9.0,14.0,21.0,24.0,26.0,29.0,29.0,28.0,26.0,28.0,28.0,30.0,29.0
10038688,26.0,27.0,29.0,29.0,28.0,31.0,35.0,36.0,37.0,38.0,37.0,36.0,37.0,36.0,35.0
10058575,9.0,11.0,10.0,9.0,11.0,13.0,14.0,15.0,17.0,19.0,20.0,23.0,26.0,24.0,26.0
10064854,28.0,30.0,31.0,43.0,49.0,65.0,81.0,100.0,121.0,140.0,146.0,139.0,116.0,84.0,54.0
10109956,26.0,23.0,22.0,27.0,45.0,49.0,53.0,51.0,44.0,50.0,65.0,64.0,67.0,62.0,54.0
10110948,10.0,5.0,4.0,6.0,5.0,2.0,3.0,5.0,4.0,2.0,4.0,3.0,3.0,6.0,9.0
10125865,9.0,10.0,11.0,16.0,18.0,18.0,17.0,17.0,18.0,17.0,14.0,11.0,12.0,14.0,12.0
10144145,26.0,27.0,24.0,22.0,19.0,16.0,16.0,18.0,20.0,17.0,16.0,11.0,10.0,11.0,15.0
10167784,11.0,10.0,13.0,16.0,15.0,15.0,16.0,17.0,15.0,15.0,16.0,19.0,21.0,24.0,22.0


In [24]:
df_ureia_15_dias_processed.isna().sum()

ureia_dia_1     3
ureia_dia_2     1
ureia_dia_3     1
ureia_dia_4     1
ureia_dia_5     1
ureia_dia_6     1
ureia_dia_7     1
ureia_dia_8     1
ureia_dia_9     1
ureia_dia_10    2
ureia_dia_11    1
ureia_dia_12    4
ureia_dia_13    3
ureia_dia_14    6
ureia_dia_15    8
dtype: int64

In [26]:
df_creatinina_15_dias_processed = preencher_valores_faltantes_linha(df_creatinina_15_dias_processed)
df_creatinina_15_dias_processed.head(10)

,creatinina_dia_1,creatinina_dia_2,creatinina_dia_3,creatinina_dia_4,creatinina_dia_5,creatinina_dia_6,creatinina_dia_7,creatinina_dia_8,creatinina_dia_9,creatinina_dia_10,creatinina_dia_11,creatinina_dia_12,creatinina_dia_13,creatinina_dia_14,creatinina_dia_15
subject_id,,,,,,,,,,,,,,,
10032381,1.3,1.2,1.2,1.2,1.2,1.2,1.2,1.2,1.2,1.2,1.0,1.0,1.1,1.2,1.2
10038688,1.6,1.9,1.9,2.0,1.5,1.5,1.6,1.6,1.8,1.6,1.4,1.3,1.3,1.3,1.1
10058575,0.8,0.7,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.7,0.6,0.6,0.6,0.5,0.6
10064854,0.7,0.6,1.1,1.3,1.4,1.8,2.2,2.9,3.2,3.6,3.7,3.3,2.5,1.7,1.1
10109956,1.6,1.5,1.4,1.6,2.3,2.1,2.2,2.0,1.8,2.0,2.1,2.1,2.1,2.0,1.9
10110948,0.6,0.5,0.4,0.5,0.5,0.4,0.5,0.5,0.5,0.4,0.5,0.5,0.4,0.4,0.4
10125865,0.7,0.7,0.6,0.8,0.8,0.8,0.8,0.9,0.9,0.7,0.7,0.6,0.7,0.7,0.6
10144145,0.8,0.7,0.6,0.5,0.5,0.5,0.5,0.6,0.5,0.5,0.5,0.5,0.5,0.5,0.5
10167784,0.7,0.6,0.6,0.6,0.7,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.5,0.5,0.5


In [27]:
df_creatinina_15_dias_processed.isna().sum()

creatinina_dia_1     2
creatinina_dia_2     1
creatinina_dia_3     1
creatinina_dia_4     1
creatinina_dia_5     1
creatinina_dia_6     1
creatinina_dia_7     1
creatinina_dia_8     1
creatinina_dia_9     1
creatinina_dia_10    1
creatinina_dia_11    0
creatinina_dia_12    1
creatinina_dia_13    0
creatinina_dia_14    2
creatinina_dia_15    0
dtype: int64

#### Deletando linhas que contém valores nulos - mesmo depois da inserção de dados artificiais

##### Ureia

In [28]:
df_creatinina_15_dias_processed[df_creatinina_15_dias_processed.isna().any(axis = 1)].index

Index([10839217, 11730347, 13784719, 15640315, 15773840, 16816440], dtype='int64', name='subject_id')

In [29]:
df_creatinina_15_dias_processed = df_creatinina_15_dias_processed.dropna(axis = 0)
df_creatinina_15_dias_processed.isna().sum().sum()

0

##### Creatinina

In [ ]:
df_creatinina_15_dias_processed[df_creatinina_15_dias_processed.isna().any(axis = 1)].index

Index([10463546, 10553635, 10996857, 11611745, 11663336, 11833476, 12344021,
       12374214, 12934260, 13681485, 13824877, 14383658, 14691603, 14816630,
       15409850, 15583807, 15640315, 15993533, 16327028, 17327554, 17484283,
       17635990, 17718694, 17803326, 18135694, 18152377, 18560897, 18638524,
       18757959],
      dtype='int64', name='subject_id')

In [30]:
df_creatinina_15_dias_processed = df_creatinina_15_dias_processed.dropna(axis = 0)
df_creatinina_15_dias_processed.isna().sum().sum()

0

### Realizando a concatenação

In [32]:
df_concatenado = concatenar_dois_datasets_por_paciente(df_esquerda = df_ureia_15_dias_processed,
                                                       df_direita = df_creatinina_15_dias_processed)

df_concatenado.head(10)

,ureia_dia_1,ureia_dia_2,ureia_dia_3,ureia_dia_4,ureia_dia_5,ureia_dia_6,ureia_dia_7,ureia_dia_8,ureia_dia_9,ureia_dia_10,...,creatinina_dia_14,creatinina_dia_15,creatinina_dia_2,creatinina_dia_3,creatinina_dia_4,creatinina_dia_5,creatinina_dia_6,creatinina_dia_7,creatinina_dia_8,creatinina_dia_9
subject_id,,,,,,,,,,,,,,,,,,,,,
10032381,9.0,9.0,9.0,14.0,21.0,24.0,26.0,29.0,29.0,28.0,...,1.2,1.2,1.2,1.2,1.2,1.2,1.2,1.2,1.2,1.2
10038688,26.0,27.0,29.0,29.0,28.0,31.0,35.0,36.0,37.0,38.0,...,1.3,1.1,1.9,1.9,2.0,1.5,1.5,1.6,1.6,1.8
10058575,9.0,11.0,10.0,9.0,11.0,13.0,14.0,15.0,17.0,19.0,...,0.5,0.6,0.7,0.5,0.5,0.5,0.5,0.5,0.5,0.5
10064854,28.0,30.0,31.0,43.0,49.0,65.0,81.0,100.0,121.0,140.0,...,1.7,1.1,0.6,1.1,1.3,1.4,1.8,2.2,2.9,3.2
10109956,26.0,23.0,22.0,27.0,45.0,49.0,53.0,51.0,44.0,50.0,...,2.0,1.9,1.5,1.4,1.6,2.3,2.1,2.2,2.0,1.8
10110948,10.0,5.0,4.0,6.0,5.0,2.0,3.0,5.0,4.0,2.0,...,0.4,0.4,0.5,0.4,0.5,0.5,0.4,0.5,0.5,0.5
10125865,9.0,10.0,11.0,16.0,18.0,18.0,17.0,17.0,18.0,17.0,...,0.7,0.6,0.7,0.6,0.8,0.8,0.8,0.8,0.9,0.9
10144145,26.0,27.0,24.0,22.0,19.0,16.0,16.0,18.0,20.0,17.0,...,0.5,0.5,0.7,0.6,0.5,0.5,0.5,0.5,0.6,0.5
10167784,11.0,10.0,13.0,16.0,15.0,15.0,16.0,17.0,15.0,15.0,...,0.5,0.5,0.6,0.6,0.6,0.7,0.6,0.6,0.6,0.6


### Verificando se concatenação gerou valores NaN

In [33]:
df_concatenado.isna().sum()

ureia_dia_1           3
ureia_dia_2           1
ureia_dia_3           1
ureia_dia_4           1
ureia_dia_5           1
ureia_dia_6           1
ureia_dia_7           1
ureia_dia_8           1
ureia_dia_9           1
ureia_dia_10          2
ureia_dia_11          1
ureia_dia_12          4
ureia_dia_13          3
ureia_dia_14          6
ureia_dia_15          8
creatinina_dia_1     77
creatinina_dia_10    77
creatinina_dia_11    77
creatinina_dia_12    77
creatinina_dia_13    77
creatinina_dia_14    77
creatinina_dia_15    77
creatinina_dia_2     77
creatinina_dia_3     77
creatinina_dia_4     77
creatinina_dia_5     77
creatinina_dia_6     77
creatinina_dia_7     77
creatinina_dia_8     77
creatinina_dia_9     77
dtype: int64

### Pegando os `id`s dos pacientes que não possuem leitura de ureia, mas possuem leitura de creatinina

In [34]:
df_concatenado[df_concatenado.isna().any(axis = 1)].index

Index([10174787, 10291098, 10719064, 10839217, 11014822, 11020519, 11052292,
       11226173, 11272213, 11340773, 11441773, 11481806, 11730347, 11821055,
       12116314, 12430647, 12619244, 12707214, 12728628, 12902491, 12911473,
       12932366, 13502902, 13573101, 13641906, 13782556, 13784719, 14217491,
       14341949, 14702642, 14756130, 14870133, 14933447, 15188471, 15193194,
       15288709, 15640315, 15640564, 15773840, 15791261, 15824431, 15983067,
       16132846, 16225498, 16244642, 16291864, 16481693, 16679905, 16816440,
       16904378, 17076438, 17229780, 17416494, 17529653, 17662799, 17790156,
       17824281, 17903930, 17918473, 18049978, 18172330, 18186302, 18399764,
       18441942, 18702320, 18858799, 19036718, 19057052, 19305085, 19392949,
       19582136, 19607985, 19666098, 19674376, 19831176, 19884808, 19894790,
       19957285],
      dtype='int64', name='subject_id')

### Removendo os pacientes que não possuem leitura de ureia, mas possuem leitura de creatinina

In [35]:
df_concatenado = df_concatenado.dropna(axis = 0)
df_concatenado.isna().sum().sum()

0

### Salvando o dataset concatenado

In [36]:
df_concatenado.to_csv("../databases/processed/ureia_creatinina_15_dias_concatenado.csv", sep = ",")